We use the same problem as our example

$$ \max  2x+3y$$
$$s.t.$$
$$2x+y<=10  $$
$$3x+6y<=40$$
$$ x,y \in Z$$

In this file, we try to implement a branch and bound procedure which only use gurobi as a LP solver.

The MIP solver will terminate (with an optimal result) when the gap between the lower and upper objective bound is less than MIPGap times the absolute value of the incumbent objective value. More precisely, if $z_{P}$ is the primal objective bound (i.e., the incumbent objective value, which is the upper bound for minimization problems), and $z_{D}$ is the dual objective bound (i.e., the lower bound for minimization problems), then the MIP gap is defined as

$$g a p=\left|z_{P}-z_{D}\right| /\left|z_{P}\right|$$

Note that if $z_{P}=z_{D}=0$, then the gap is defined to be zero. If $z_{P}=0$ and $z_{D} \neq 0$, the gap is defined to be infinity.
For most models, $z_{P}$ and $z_{D}$ will have the same sign throughout the optimization process, and then the gap is monotonically decreasing. But if $z_{P}$ and $z_{D}$ have opposite signs, the relative gap may increase after finding a new incumbent solution, even though the absolute gap $\left|z_{P}-z_{D}\right|$ has decreased.

In [ ]:
# -*- coding: utf-8 -*-
"""
Spyder Editor

This is a temporary script file.
"""


import gurobipy as gp
from gurobipy import GRB
import math
import copy

n=gp.Model()
x={}
x[0] = n.addVar(lb=0, vtype=GRB.CONTINUOUS, name="x1")
x[1] = n.addVar(lb=0, vtype=GRB.CONTINUOUS , name ="x2")
n.setObjective(2*x[0]+3*x[1], GRB.MAXIMIZE) 
n.addConstr(2*x[0]+x[1]<=10)
n.addConstr(3*x[0]+6*x[1]<=40)
n.setParam('OutputFlag', 0)
print('#' * 70)

In [3]:
global UB

UB=0

def integer_Branch_and_Bound(n):
    global UB
    n.optimize()
    try:
        LP_Obj=n.ObjVal
        LP_relaxation = [n.getVars()[i].x for i in range(len(n.getVars()))]
        
        if LP_Obj<UB:
            print("prune node %s" %LP_relaxation)
            return 0,[0,0]
        elif not any(divmod(n.getVars()[i].x,1)[1] for i in range(len(n.getVars()))):
            UB=LP_Obj if LP_Obj>UB else UB
            print('LP_relaxation_node: %s' %LP_relaxation )
            print('LP_objective: %s' %LP_Obj)
            print('Lower_bound: %s' %UB)
            print('#' * 70)
            return LP_Obj, LP_relaxation
        else:
            print('LP_relaxation_node: %s' %LP_relaxation )
            print('LP_objective: %s' %LP_Obj)
            print('Lower_bound: %s' %UB)
            print('#' * 70)
            ind = [i for i, x in enumerate(LP_relaxation) if x-math.floor(x)>1e-5 ][0]
            n_left=n.copy()
            n_right=n.copy()
            del n
            n_left.addConstr(n_left.getVars()[ind]<=math.floor(LP_relaxation[ind]))
            n_right.addConstr(n_right.getVars()[ind]>=math.ceil(LP_relaxation[ind]))
            LP_Obj_left,LP_relaxation_left=integer_Branch_and_Bound(n_left)
            LP_Obj_right,LP_relaxtion_right=integer_Branch_and_Bound(n_right)
            if LP_Obj_left>LP_Obj_right:
                return LP_Obj_left, LP_relaxation_left
            else:
                return LP_Obj_right, LP_relaxtion_right
    except:
        return 0,[0,0]
    
        
print(integer_Branch_and_Bound(n))

Using license file c:\gurobi911\gurobi.lic
######################################################################
LP_relaxation_node: [2.2222222222222223, 5.555555555555555]
LP_objective: 21.111111111111107
Lower_bound: 0
######################################################################
LP_relaxation_node: [2.0, 5.666666666666667]
LP_objective: 21.0
Lower_bound: 0
######################################################################
LP_relaxation_node: [2.0, 5.0]
LP_objective: 19.0
Lower_bound: 19.0
######################################################################
LP_relaxation_node: [1.3333333333333333, 6.0]
LP_objective: 20.666666666666668
Lower_bound: 19.0
######################################################################
LP_relaxation_node: [1.0, 6.166666666666667]
LP_objective: 20.5
Lower_bound: 19.0
######################################################################
LP_relaxation_node: [1.0, 6.0]
LP_objective: 20.0
Lower_bound: 20.0
#############################